## Tensorflow Serving Version 1.13 Example

Train and export a simple Softmax Regression TensorFlow model.
The model is from the TensorFlow "MNIST For ML Beginner" tutorial. This program
simply follows all its training instructions, and uses TensorFlow SavedModel to
export the trained model with proper signatures that can be loaded by standard
tensorflow_model_server.
<br>
Usage: mnist_saved_model.py [--training_iteration=x] [--model_version=y] export_dir

In [6]:
from __future__ import print_function

import os
import sys
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.contrib.session_bundle import exporter

In [2]:
tf.app.flags.DEFINE_integer('training_epochs', 500, 'number of training epochs.')
tf.app.flags.DEFINE_integer('training_iteration', 1000, 'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('work_dir', './', 'Working directory.')
FLAGS = tf.app.flags.FLAGS
export_path_base = ('./model')
batch_size = 50

In [3]:
# training data
train_height = np.array([1.60, 1.62, 1.68, 1.69, 1.70, 1.70, 1.71, 1.72, 1.80, 1.76, 1.77, 1.78, 1.77, 1.78, 1.79, 1.80, 1.81, 1.82, 1.83, 1.94])
train_weight = np.array([60.5, 58.8, 62.7, 62.8, 63.1, 58.9, 61.5, 65.8, 70.0, 65.8, 66.8, 68.0, 66.7, 60.8, 72.4, 70.5, 72.8, 75.8, 74.6, 83.2])

In [4]:
def main(_): 
    k = tf.Variable(0., name='k')
    b = tf.Variable(0., name='b')
    
    serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
    feature_configs = {"x": tf.FixedLenFeature([1], dtype=tf.float32),}
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    
    x = tf.identity(tf_example["x"], name="x")
    y = tf.add(tf.multiply(k, x), b, name="y")
    
    prediction = tf.add(tf.multiply(x, k), b, name='results')
    
    save = tf.train.Saver(
        {
            "k": k,
            "b": b
        },
        sharded=True,
        write_version=tf.train.SaverDef.V2 if use_checkpoint_v2 else
        tf.train.SaverDef.V1
    )
    
    # Create a RegressionSignature for our input and output.
    regression_signature = exporter.regression_signature(
        input_tensor=serialized_tf_example,
        output_tensor=tf.identity(y)
    )
    
    named_graph_signature = {
        "inputs": exporter.generic_signature({"x": x}),
        "outputs": exporter.generic_signature({"y": y})
    }
    
    cost = tf.reduce_mean(tf.square(y - prediction))
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
    init = tf.global_variables_initializer()

    display = 50

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(FLAGS.training_epochs):
            for(x_, y_) in zip(train_height, train_weight):
                sess.run(optimizer, {x: train_height, y: train_weight})

            if epoch % display == 0:
                loss = sess.run(cost, {x: train_height, y: train_weight})
                print("Epoch: ", epoch, " cost: ", loss, " w: ", sess.run(k), " b: ", sess.run(b))

        print('Done training!')
        
        export_path = os.path.join(
            tf.compat.as_bytes(export_path_base),
            tf.compat.as_bytes(str(FLAGS.model_version)))

        print('Exporting trained model to', export_path)
        
        builder = tf.saved_model.builder.SavedModelBuilder(export_path)
        
        regression_inputs = tf.saved_model.utils.build_tensor_info(serialized_tf_example)
        regression_outputs = tf.saved_model.utils.build_tensor_info(y)

        tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
        tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

        prediction_signature = (
            tf.saved_model.signature_def_utils.build_signature_def(
                inputs={'height': tensor_info_x},
                outputs={'weight': tensor_info_y},
                method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
            )
        )
        
        print('tensor_info_x: ', tensor_info_x)
        print('tensor_info_y: ', tensor_info_y)
        print('prediction_signature: ', prediction_signature)
        print('method_name: ', tf.saved_model.signature_constants.PREDICT_METHOD_NAME)

        builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                'predict_weight':
                  prediction_signature,
            })

        builder.save()

        print('Done exporting!')

In [5]:
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch:  0  cost:  19.287436  w:  29.172255  b:  16.031004
Epoch:  50  cost:  14.432498  w:  40.040615  b:  -3.0536795
Epoch:  100  cost:  11.676342  w:  48.229496  b:  -17.433226
Epoch:  150  cost:  10.111669  w:  54.39948  b:  -28.26763
Epoch:  200  cost:  9.2233925  w:  59.048317  b:  -36.43092
Epoch:  250  cost:  8.719124  w:  62.55103  b:  -42.581627
Epoch:  300  cost:  8.432861  w:  65.19013  b:  -47.21586
Epoch:  350  cost:  8.270343  w:  67.17856  b:  -50.707497
Epoch:  400  cost:  8.178076  w:  68.676796  b:  -53.338387
Epoch:  450  cost:  8.125703  w:  69.80563  b:  -55.320606
Done training!
Exporting trained model to b'./model/1'
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
tensor_info_x:  name: "x:0"
dtyp

SystemExit: 

/home/bai/.virtualenvs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Start Tensorflow Serving (don't specify model version)

docker run -p 8501:8501 --mount type=bind,source=/home/bai/PycharmProjects/tensorflow_for_enginner/11_tf_serving/model/1,target=/models/my_model -e MODEL_NAME=my_model -t tensorflow/serving